In [8]:
import requests
from bs4 import BeautifulSoup
# Define a function to scrape information from Wikipedia
def scrape_wikipedia(subject):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        # Get the Wikipedia page for the subject
        page = requests.get(f'https://en.wikipedia.org/wiki/{subject}')

        print(page)
        # Create a BeautifulSoup object to parse the HTML content
        soup = BeautifulSoup(page.content, "html.parser")
        
        # Find the title of the Wikipedia page
        title = soup.find("h1", {"id": "firstHeading"}).text
        # print(title)
        content = ""
        for element in soup.find_all():
            # Ignore script and style tags
            if element.name in ['script', 'style', 'nav', 'table']:
                continue
            # Extract text from headings, paragraphs, and list items
            elif element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']:
                content += element.get_text() + " "
        # print(content)
        return content
    except:
        return None

In [9]:

dataset = scrape_wikipedia("president of the us?")
print(dataset)

<Response [200]>
Contents President of the United States 
  
 The president of the United States (POTUS)[A] is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
 The power of the presidency has grown substantially[11] since the first president, George Washington, took office in 1789.[6] While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, with notable expansion during the presidency of Franklin D. Roosevelt. In modern times, the president is one of the world's most powerful political figures – the leader of the only remaining global superpower.[12][13][14][15] As the leader of the nation with the largest economy by nominal GDP, the president possesses significant domestic and international hard and sof

In [14]:
!pip install transformers

import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Define the paragraph and the question
dataset = "The United States of America (USA), commonly known as the United States (U.S. or US) or America, is a country primarily located in North America. It consists of 50 states, a federal district, five major self-governing territories, and various possessions. The United States has a population of over 330 million people, making it the third most populous country in the world. It is a federal republic and a representative democracy, with a president as the head of state and a vice president as the second highest position in the executive branch."

question = "Who is the president of the United States?"

# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = AutoModelForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Tokenize the input
inputs = tokenizer(question, dataset[0:10000], return_tensors='pt')
start_scores, end_scores = model(**inputs)

# Find the start and end positions with the highest scores
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores) + 1

# Decode the answer
answer_tokens = inputs['input_ids'][0][start_index:end_index]
answer = tokenizer.decode(answer_tokens)

# Print the answer
print(answer)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str

In [16]:
import nltk
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load NLTK data
nltk.download('punkt')

# Load Hugging Face models and tokenizer
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Define a function to retrieve the answer for a given question
def get_answer(question):
    # Tokenize the question and add special tokens
    inputs = tokenizer(question, dataset[0:1000], return_tensors="pt")
    
    # Pass the inputs to the model to generate a prediction
    outputs = model(**inputs)
    
    # Extract the start and end indices of the predicted answer
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1
    
    # Decode the predicted answer
    answer_tokens = inputs["input_ids"][0][answer_start:answer_end]
    answer = tokenizer.decode(answer_tokens)
    
    return answer

# Test the chatbot
while True:
    user_input = input("You: ")
    answer = get_answer(user_input)
    
    if not answer:
        print("ChatBot: I'm sorry, I don't understand your question.")
    else:
        print("ChatBot:", answer)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nikhilpatil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


You:  who is the ceo of microsoft?


ChatBot: president


You:  who is the current president?


ChatBot: the head of state


You:  what is your name?


ChatBot: the united states of america


KeyboardInterrupt: Interrupted by user